# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

In [ ]:
# Sparkify - Data Engineer Nanodegree program Cassandra Data Model Project
# By JGEL
# February 2020

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
#print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
# set keyspace
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
## Table creation
# Table for question number 1, artist, song and song's length. Get sessionId -> 338 and itemInSession -> 4
query = "CREATE TABLE IF NOT EXISTS song_session"
query = query + "(session_id int, item_in_session int, artist text, song_title text, song_length decimal, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
# Table for question number 2, artist, sorted song and user for userid -> 10 and sessionid -> 182
query = "CREATE TABLE IF NOT EXISTS song_user_session"
query = query + "(user_id int, session_id int, song_title text, item_in_session int, artist text, last_name text, first_name text, PRIMARY KEY ((user_id, session_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

# Table for question number 3. Every user name (full) in music app who listened the song <<All hands against his own>>
query = "CREATE TABLE IF NOT EXISTS user_song_title"
query = query + "(song_title text, user_id int, last_name text, first_name text, PRIMARY KEY ((song_title, user_id)))"
try:
    session.execute(query)
except Exception as e:
    print(e)                 

In [9]:
## Insert data statements
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query1 = "INSERT INTO song_session (session_id, item_in_session, artist, song_title, song_length)"
        query1 = query1 + "VALUES (%s, %s, %s, %s, %s)"
        query2 = "INSERT INTO song_user_session (user_id, session_id, song_title, item_in_session, artist, last_name, first_name)"
        query2 = query2 + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        query3 = "INSERT INTO user_song_title (song_title, user_id, last_name, first_name)"
        query3 = query3 + "VALUES (%s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query1, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        session.execute(query2, (int(line[10]), int(line[8]), line[9], int(line[3]), line[0], line[4], line[1]))
        session.execute(query3, (line[9], int(line[10]), line[4], line[1]))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## First question -- Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4 --
query = "SELECT artist, song_title, song_length FROM song_session WHERE session_id=338 AND item_in_session=4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.3073


In [11]:
## Second Question -- Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182 --
query = "SELECT artist, song_title, first_name, last_name FROM song_user_session WHERE user_id=10 AND session_id=182"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song_title, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [13]:
## Third Question -- Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own' --
query = "SELECT user_id, first_name, last_name FROM user_song_title WHERE song_title = 'All Hands Against His Own' ALLOW FILTERING"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.user_id, row.first_name, row.last_name)

80 Tegan Levine
95 Sara Johnson
29 Jacqueline Lynch


### Drop the tables before closing out the sessions

In [14]:
## Dropping Tables

query = "DROP TABLE IF EXISTS song_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS song_user_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS user_song_title"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [15]:
session.shutdown()
cluster.shutdown()